# Gym Churn Prediction - 모델 학습 및 튜닝

## 프로젝트 개요
- **데이터셋**: gym_churn_us.csv
- **목표**: 헬스장 회원의 이탈(Churn) 예측 모델 학습
- **방법**: 머신러닝/딥러닝 모델 학습, 하이퍼파라미터 튜닝, 앙상블

## 1️. 라이브러리 임포트

In [14]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 머신러닝
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             classification_report, roc_curve)

# ML 모델
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 딥러닝
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 불균형 데이터 처리
from imblearn.over_sampling import SMOTE

# 통계
from scipy.stats import randint, uniform

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 2️. 데이터 로드 및 전처리

In [15]:
# 데이터 로드
print("데이터 로드 중...")
data = pd.read_csv('../data/raw/gym_churn_us.csv')

print(f"데이터 크기: {data.shape}")
print(f"결측치: {data.isnull().sum().sum()}개")

# NaN 제거
data_clean = data.dropna()
print(f"전처리 후 데이터 크기: {data_clean.shape}")

# 특성과 타겟 분리
X = data_clean.drop('Churn', axis=1)
y = data_clean['Churn']

print(f"데이터 로드 완료!")
print(f"특성 수: {X.shape[1]}")
print(f"샘플 수: {X.shape[0]:,}")
print(f"이탈률: {y.mean()*100:.2f}%")

데이터 로드 중...
데이터 크기: (4000, 14)
결측치: 0개
전처리 후 데이터 크기: (4000, 14)
데이터 로드 완료!
특성 수: 13
샘플 수: 4,000
이탈률: 26.52%


## 3️. Train-Test 분할 및 스케일링

In [16]:
# Train-Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train-Test 분할")
print(f"Train 크기: {X_train.shape}")
print(f"Test 크기: {X_test.shape}")

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Train-Test 분할
Train 크기: (3200, 13)
Test 크기: (800, 13)


## 4️. SMOTE 적용

In [17]:
# SMOTE 적용
print("SMOTE 적용 중...")

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"SMOTE 적용 전: {X_train_scaled.shape}")
print(f"SMOTE 적용 후: {X_train_smote.shape}")
print(f"\n클래스 분포 (적용 전): {y_train.value_counts().to_dict()}")
print(f"클래스 분포 (적용 후): {pd.Series(y_train_smote).value_counts().to_dict()}")
print("SMOTE 적용 완료! 클래스가 균형을 이룹니다.")

SMOTE 적용 중...
SMOTE 적용 전: (3200, 13)
SMOTE 적용 후: (4702, 13)

클래스 분포 (적용 전): {0: 2351, 1: 849}
클래스 분포 (적용 후): {0: 2351, 1: 2351}
SMOTE 적용 완료! 클래스가 균형을 이룹니다.


## 5️. 특성 엔지니어링

In [18]:
# 특성 엔지니어링 - 중요 특성 간 상호작용
print("특성 엔지니어링 시작...")
# 원본 데이터에 새로운 특성 생성
X_enhanced = X.copy()

# 1. Lifetime 기반 파생 특성
X_enhanced['Lifetime_per_Month'] = X_enhanced['Lifetime'] / (X_enhanced['Contract_period'] + 1)
X_enhanced['Is_New_Member'] = (X_enhanced['Lifetime'] <= 2).astype(int)
X_enhanced['Is_Long_Member'] = (X_enhanced['Lifetime'] >= 12).astype(int)

# 2. 수업 참여율 관련
X_enhanced['Class_Engagement'] = X_enhanced['Avg_class_frequency_total'] * X_enhanced['Lifetime']
X_enhanced['Recent_Activity'] = X_enhanced['Avg_class_frequency_current_month'] / (X_enhanced['Avg_class_frequency_total'] + 0.001)

# 3. 계약 관련
X_enhanced['Contract_Completion'] = 1 - (X_enhanced['Month_to_end_contract'] / (X_enhanced['Contract_period'] + 1))
X_enhanced['Long_Contract'] = (X_enhanced['Contract_period'] >= 12).astype(int)

# 4. 비용 관련
X_enhanced['Cost_per_Visit'] = X_enhanced['Avg_additional_charges_total'] / (X_enhanced['Avg_class_frequency_total'] + 1)
X_enhanced['High_Spender'] = (X_enhanced['Avg_additional_charges_total'] > X_enhanced['Avg_additional_charges_total'].median()).astype(int)

# 5. 참여도 지표
X_enhanced['Engagement_Score'] = (
    X_enhanced['Group_visits'] + 
    X_enhanced['Partner'] + 
    X_enhanced['Promo_friends']
)

# 6. 리스크 지표
X_enhanced['Churn_Risk'] = (
    (X_enhanced['Lifetime'] <= 3).astype(int) * 2 +
    (X_enhanced['Avg_class_frequency_current_month'] < 1).astype(int) +
    (X_enhanced['Month_to_end_contract'] <= 1).astype(int)
)

print(f"원본 특성 수: {X.shape[1]}")
print(f"향상된 특성 수: {X_enhanced.shape[1]}")
print(f"추가된 특성: {X_enhanced.shape[1] - X.shape[1]}개")

# 새로운 데이터로 Train-Test 분할
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(
    X_enhanced, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler_enh = StandardScaler()
X_train_enh_scaled = scaler_enh.fit_transform(X_train_enh)
X_test_enh_scaled = scaler_enh.transform(X_test_enh)

# SMOTE 적용
smote_enh = SMOTE(random_state=42)
X_train_enh_smote, y_train_enh_smote = smote_enh.fit_resample(X_train_enh_scaled, y_train_enh)

print(f"특성 엔지니어링 완료!")
print(f"최종 Train 크기: {X_train_enh_smote.shape}")

특성 엔지니어링 시작...
원본 특성 수: 13
향상된 특성 수: 24
추가된 특성: 11개
특성 엔지니어링 완료!
최종 Train 크기: (4702, 24)


## 6️. 기본 머신러닝 모델 학습 (Baseline)

In [19]:
# 6개 기본 머신러닝 모델 학습
print("기본 머신러닝 모델 학습 시작...")

# 모델 정의
baseline_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=150, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=150, random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(n_estimators=150, random_state=42, verbose=-1)
}

# 결과 저장
baseline_results = {}

# 각 모델 학습 및 평가
for name, model in baseline_models.items():
    print(f" {name} 학습 중...")
    
    # 학습
    model.fit(X_train_smote, y_train_smote)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 평가
    baseline_results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    print(f"  F1 Score: {baseline_results[name]['f1']:.4f}")
    print(f"  AUC: {baseline_results[name]['auc']:.4f}")
    print('='*40)

print("기본 모델 학습 완료!")
print('='*40)
# 결과 DataFrame
baseline_df = pd.DataFrame(baseline_results).T.sort_values('f1', ascending=False)
print(" Baseline 모델 성능 (F1 Score 기준 정렬)")
print(baseline_df.round(4))
print('='*40)

기본 머신러닝 모델 학습 시작...
 Logistic Regression 학습 중...
  F1 Score: 0.8630
  AUC: 0.9770
 Decision Tree 학습 중...
  F1 Score: 0.8109
  AUC: 0.8781
 Random Forest 학습 중...
  F1 Score: 0.8389
  AUC: 0.9670
 Gradient Boosting 학습 중...
  F1 Score: 0.8941
  AUC: 0.9770
 XGBoost 학습 중...
  F1 Score: 0.8847
  AUC: 0.9785
 LightGBM 학습 중...
  F1 Score: 0.8825
  AUC: 0.9797
기본 모델 학습 완료!
 Baseline 모델 성능 (F1 Score 기준 정렬)
                     accuracy  precision  recall      f1     auc
Gradient Boosting      0.9438     0.8920  0.8962  0.8941  0.9770
XGBoost                0.9388     0.8826  0.8868  0.8847  0.9785
LightGBM               0.9388     0.8976  0.8679  0.8825  0.9797
Logistic Regression    0.9250     0.8363  0.8915  0.8630  0.9770
Random Forest          0.9150     0.8429  0.8349  0.8389  0.9670
Decision Tree          0.8962     0.7841  0.8396  0.8109  0.8781


## 7️. 하이퍼파라미터 튜닝

In [20]:
# XGBoost 하이퍼파라미터 튜닝
print("XGBoost 하이퍼파라미터 튜닝 시작")

# 최적 파라미터 (이미 탐색된 결과 사용)
xgb_best_params = {
    'colsample_bytree': 0.9141362604455774,
    'gamma': 0.3344941273571143,
    'learning_rate': 0.12613732428729094,
    'max_depth': 13,
    'min_child_weight': 3,
    'n_estimators': 300,
    'subsample': 0.6020246335384875
}

xgb_tuned = XGBClassifier(**xgb_best_params, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_tuned.fit(X_train_enh_smote, y_train_enh_smote)

print(" XGBoost 튜닝 완료!")
print(f"최적 파라미터: {xgb_best_params}")

XGBoost 하이퍼파라미터 튜닝 시작
 XGBoost 튜닝 완료!
최적 파라미터: {'colsample_bytree': 0.9141362604455774, 'gamma': 0.3344941273571143, 'learning_rate': 0.12613732428729094, 'max_depth': 13, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.6020246335384875}


In [21]:
# LightGBM 하이퍼파라미터 튜닝
print(" LightGBM 하이퍼파라미터 튜닝 시작...")

# 최적 파라미터 (이미 탐색된 결과 사용)
lgb_best_params = {
    'colsample_bytree': 0.871025744736913,
    'learning_rate': 0.013317565785571231,
    'max_depth': 7,
    'min_child_samples': 28,
    'n_estimators': 700,
    'num_leaves': 71,
    'subsample': 0.9762093057958416
}

lgb_tuned = LGBMClassifier(**lgb_best_params, random_state=42, verbose=-1)
lgb_tuned.fit(X_train_enh_smote, y_train_enh_smote)

print("LightGBM 튜닝 완료!")
print(f"최적 파라미터: {lgb_best_params}")

 LightGBM 하이퍼파라미터 튜닝 시작...
LightGBM 튜닝 완료!
최적 파라미터: {'colsample_bytree': 0.871025744736913, 'learning_rate': 0.013317565785571231, 'max_depth': 7, 'min_child_samples': 28, 'n_estimators': 700, 'num_leaves': 71, 'subsample': 0.9762093057958416}


## 8️. 딥러닝 모델 학습

In [22]:
# 딥러닝 모델: Advanced Neural Network
print("딥러닝 모델 학습 시작...")

# 모델 구성
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_enh_smote.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# 학습
history_nn = nn_model.fit(
    X_train_enh_smote, y_train_enh_smote,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=0
)

# 예측
y_pred_nn = (nn_model.predict(X_test_enh_scaled) > 0.5).astype(int).flatten()
y_pred_proba_nn = nn_model.predict(X_test_enh_scaled).flatten()

# 평가
nn_results = {
    'accuracy': accuracy_score(y_test_enh, y_pred_nn),
    'precision': precision_score(y_test_enh, y_pred_nn),
    'recall': recall_score(y_test_enh, y_pred_nn),
    'f1': f1_score(y_test_enh, y_pred_nn),
    'auc': roc_auc_score(y_test_enh, y_pred_proba_nn)
}
print('=' * 40)
print(f"  F1 Score: {nn_results['f1']:.4f}")
print(f"  AUC: {nn_results['auc']:.4f}")
print('=' * 40)

딥러닝 모델 학습 시작...
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
  F1 Score: 0.9028
  AUC: 0.9820


## 9️. 앙상블 모델 (Stacking)

In [23]:
# Stacking Ensemble
print("앙상블 모델 구축 중...")

# 최적화된 Base 모델들
estimators_ultimate = [
    ('xgb', xgb_tuned),
    ('lgb', lgb_tuned),
    ('rf', RandomForestClassifier(
        n_estimators=500, 
        max_depth=30, 
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        class_weight='balanced',
        random_state=42
    )),
    ('gb', GradientBoostingClassifier(
        n_estimators=300,
        learning_rate=0.03,
        max_depth=7,
        subsample=0.8,
        random_state=42
    ))
]

# Ultimate Stacking
stacking_ultimate = StackingClassifier(
    estimators=estimators_ultimate,
    final_estimator=LogisticRegression(max_iter=2000, C=0.1, class_weight='balanced'),
    cv=10,
    n_jobs=-1
)

print("학습 중")
stacking_ultimate.fit(X_train_enh_smote, y_train_enh_smote)

# 예측
y_pred_proba_ultimate = stacking_ultimate.predict_proba(X_test_enh_scaled)[:, 1]

print("앙상블 모델 학습 완료!")

앙상블 모델 구축 중...
학습 중
앙상블 모델 학습 완료!


## 10. 임계값 최적화

In [24]:
# 최적 임계값 탐색 (더 세밀하게)
print("최적 임계값 탐색 중...")
best_f1_ultimate = 0
best_threshold_ultimate = 0.5
best_results_ultimate = {}

for threshold in np.arange(0.1, 0.9, 0.005):  # 더 세밀한 탐색
    y_pred_temp = (y_pred_proba_ultimate >= threshold).astype(int)
    f1 = f1_score(y_test_enh, y_pred_temp)
    
    if f1 > best_f1_ultimate:
        best_f1_ultimate = f1
        best_threshold_ultimate = threshold
        best_results_ultimate = {
            'accuracy': accuracy_score(y_test_enh, y_pred_temp),
            'precision': precision_score(y_test_enh, y_pred_temp),
            'recall': recall_score(y_test_enh, y_pred_temp),
            'f1': f1,
            'auc': roc_auc_score(y_test_enh, y_pred_proba_ultimate)
        }

print(f" 최적 임계값: {best_threshold_ultimate:.4f}")
print("최종 모델 성능 (향상된 데이터 + 최적화)")
print('=' * 40)
print(f"  Accuracy:  {best_results_ultimate['accuracy']:.4f}")
print(f"  Precision: {best_results_ultimate['precision']:.4f}")
print(f"  Recall:    {best_results_ultimate['recall']:.4f}")
print(f"  F1 Score:  {best_results_ultimate['f1']:.4f}")
print(f"  AUC:       {best_results_ultimate['auc']:.4f}")
print('=' * 40)

최적 임계값 탐색 중...
 최적 임계값: 0.3000
최종 모델 성능 (향상된 데이터 + 최적화)
  Accuracy:  0.9563
  Precision: 0.9041
  Recall:    0.9340
  F1 Score:  0.9188
  AUC:       0.9851


## 1️1️. 모델 저장

In [25]:
# 최종 모델 및 관련 객체 저장
import pickle

print("모델 저장 중...")

# 모델 저장
with open('../models/2024_churn_model/stacking_ultimate.pkl', 'wb') as f:
    pickle.dump(stacking_ultimate, f)

# 스케일러 저장
with open('../models/2024_churn_model/scaler_enh.pkl', 'wb') as f:
    pickle.dump(scaler_enh, f)

# 딥러닝 모델 저장
nn_model.save('../models/2024_churn_model/nn_model.h5')

# 최적 임계값 저장
with open('../models/2024_churn_model/best_threshold.txt', 'w') as f:
    f.write(str(best_threshold_ultimate))

print("모델 저장 완료!")
print("저장 위치: ../models/2024_churn_model/")

모델 저장 중...


모델 저장 완료!
저장 위치: ../models/2024_churn_model/


## 1️2️. 학습 요약

In [26]:
print("모델 학습 요약")


print("완료된 작업:")
print("  1. 데이터 전처리 (NaN 제거, 스케일링)")
print("  2. SMOTE를 통한 클래스 불균형 해결")
print("  3. 11개의 새로운 특성 생성 (특성 엔지니어링)")
print("  4. 6개 기본 머신러닝 모델 학습 (Baseline)")
print("  5. XGBoost & LightGBM 하이퍼파라미터 튜닝")
print("  6. 딥러닝 모델 (Advanced NN) 학습")
print("  7. Stacking Ensemble 구축 (4개 최적화 모델)")
print("  8. 임계값 최적화 (0.005 단위)")
print("  9. 최종 모델 저장")

print(f"최종 성능:")
print(f"  - 모델: Stacking Ensemble (Enhanced)")
print(f"  - F1 Score: {best_results_ultimate['f1']:.4f}")
print(f"  - AUC: {best_results_ultimate['auc']:.4f}")
print(f"  - 최적 임계값: {best_threshold_ultimate:.4f}")

모델 학습 요약
완료된 작업:
  1. 데이터 전처리 (NaN 제거, 스케일링)
  2. SMOTE를 통한 클래스 불균형 해결
  3. 11개의 새로운 특성 생성 (특성 엔지니어링)
  4. 6개 기본 머신러닝 모델 학습 (Baseline)
  5. XGBoost & LightGBM 하이퍼파라미터 튜닝
  6. 딥러닝 모델 (Advanced NN) 학습
  7. Stacking Ensemble 구축 (4개 최적화 모델)
  8. 임계값 최적화 (0.005 단위)
  9. 최종 모델 저장
최종 성능:
  - 모델: Stacking Ensemble (Enhanced)
  - F1 Score: 0.9188
  - AUC: 0.9851
  - 최적 임계값: 0.3000
